# 1. 파인 튜닝
- 1_web_crawl.ipynb에서 얻었던 data.txt를 KoGPT2의 fine tuning을 해 학습시켜보았다. 그리고 이를 불러와 학습에 활용해보았다.  [다음의 레포지토리](https://github.com/NLP-kr/tensorflow-ml-nlp-tf2)를 참조하였다.

In [ ]:
#필요한 것들 설치
!pip install gluonnlp

In [ ]:
!pip install mxnet

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
#라이브러리 import
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel
import tensorflow as tf
import numpy as np
import os
from nltk.tokenize import sent_tokenize

In [ ]:
#모델 생성 방식 따라 디렉토리 경로 입력하는 클래스
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [ ]:
#파라미터 불러오는 명령어
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

In [ ]:
#모델 위치 설정 및 불러오기
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#토크나이저
BATCH_SIZE = 16
NUM_EPOCHS = 500
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [ ]:
#토크나이저 & 사전 학습 모델 통해 문장 만들기 함수
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [ ]:
#가사 집어넣기
DATA_IN_PATH = './data_in/KOR/' #colab에서 넣는데 사용한 주소
TRAIN_DATA_FILE = 'data.txt' #1_web_crawl.inpyb에서 만들어진 data.txt와 동일함

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]
sents

['어린 햇살 아래서 뛰어놀곤 했었던 가쁜 숨결 굽이진 골목 지나 길을 따라가보면 같은 기억 어른이란 시간은 아직 어색하게도 나를 채워 많은 게 변했다 해 여긴 그대로인걸 youll feel the same 땀에 젖어 놀았던 우리는 너와 난 이젠 돌아갈 순 없지만 낡아진 서랍 속에서 작았던 서롤 기억해 when im far from home always 떠올라 난 아직도 반짝이던 네 두 눈 마주 보던 그림자 마주 보던 우리는 여기 still same 시간은 언제나 날 울리는 존재지만 놓질 못해 많은 게 더 지날 땐 여긴 또 하나의 our home 그땐 우린 어딜까 낡아진 서랍 속에서 작았던 서롤 기억해 when im far from home always 떠올라 난 아직도 반짝이던 네 두 눈 혼자서 숨겼던 널 향한 마음은 알게 하진 않을 거야 널 볼 수 있다면 그걸로 충분해 담을게 두 눈에 언제든 항상 낡아진 서랍 속에서 작았던 서롤 기억해 when im far from home always 떠올라 난 아직도 반짝이던 네 두 눈어제 너는 나를 버렸어 나는 아무 변명하지 못하고 얌전하게 집에 돌아와 너무 피곤해 잠이 들었어 눈이 떠지자마자 정신이 없지 지각은 말이 안 돼 출근해야지 시간이 모자라 널 생각하고 아파하기엔 내가 너무 바빠 눈물이 맺혔을지도 아닌가 졸린건지도 어쩌면 널 좋아하지 않았었나봐 연락을 기다릴지도 아닌가 귀찮을지도 어쩌면 널 사랑하진 않았었나봐 이제 나는 너를 잊었어 생각해 보니 오늘 하루 종일 네 생각이 나질 않았고 왠지 웃으며 잠이 들었어 이별이 항상 지독할 필요는 없지 우리도 각자 가던 길을 가야지 못다 한 마음도 전하지 못한 말도 많았지만 내가 너무 바빠 눈물이 맺혔을지도 아닌가 졸린건지도 어쩌면 널 좋아하지 않았었나봐 연락을 기다릴지도 아닌가 귀찮을지도 어쩌면 널 사랑하진 않았었나봐 아냐 그랬을 리가 없지 너 없인 살 수 없었던 꿈같은 날들이 있었지 나를 안아주던 숨결도 빛이 나던 그 입술도 시간을 되돌려 갈 수 있다 해도 이젠 너무 

In [ ]:
#토크나이저에 넣기
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

In [ ]:
#입출력 구성
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = tf.keras.preprocessing.sequence.pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [ ]:
#손실함수 & 정확도 측정 설정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [ ]:
#모델 컴파일
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [ ]:
#학습 실행
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

Epoch 1/500
1/1 [==============================] - 22s 22s/step - loss: 5.1721 - accuracy_function: 0.1667
Epoch 2/500
1/1 [==============================] - 1s 1s/step - loss: 4.4711 - accuracy_function: 0.2167
Epoch 3/500
1/1 [==============================] - 1s 1s/step - loss: 3.6425 - accuracy_function: 0.2889
Epoch 4/500
1/1 [==============================] - 1s 1s/step - loss: 2.9444 - accuracy_function: 0.3333
Epoch 5/500
1/1 [==============================] - 1s 1s/step - loss: 2.5432 - accuracy_function: 0.3600
Epoch 6/500
1/1 [==============================] - 1s 1s/step - loss: 1.9976 - accuracy_function: 0.4056
Epoch 7/500
1/1 [==============================] - 1s 1s/step - loss: 1.4848 - accuracy_function: 0.4619
Epoch 8/500
1/1 [==============================] - 1s 1s/step - loss: 1.0917 - accuracy_function: 0.5000
Epoch 9/500
1/1 [==============================] - 1s 1s/step - loss: 0.7355 - accuracy_function: 0.5407
Epoch 10/500
1/1 [==============================] - 1

In [ ]:
#저장
DATA_OUT_PATH = './data_out'
model_name = 'tf2_gpt2_finetuned_model'
save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#결과 확인
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, greedy=True))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=0, top_p=0.95))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=0, top_p=0.90))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=0, top_p=0.7))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=0, top_p=0.5))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=5, top_p=0.))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=10, top_p=0.))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=50, top_p=0.))
print('-'*32)
print(generate_sent('너를 생각하는 오늘의 난', gpt_model, top_k=100, top_p=0.))
print('-'*32)

너를 생각하는 오늘의 난 행복하답니다 햇살이 따사로운 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난 역시 외로워요 햇살이 따사로운 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난도 오늘처럼 말해요 햇살이 따사로운 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난 너무 행복하단다 내 말은 그렇지 그런데가 이게 뭐 하는 짓인가 제발 날 좀 믿어 봐
--------------------------------
너를 생각하는 오늘의 난 너무도 외로운 사람아 너의 말대로 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난 잔다
--------------------------------
너를 생각하는 오늘의 난 참 바보 같네.
--------------------------------
너를 생각하는 오늘의 난 너무도 외로워요 햇살이 따사로운 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난 그냥 그대 돌아오세
--------------------------------
너를 생각하는 오늘의 난 아직 너와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------


In [ ]:
#결과 확인
print(generate_sent('사랑해', gpt_model, greedy=True))
print('-'*32)
print(generate_sent('사랑해', gpt_model))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=0, top_p=0.95))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=0, top_p=0.90))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=0, top_p=0.7))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=0, top_p=0.5))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=5, top_p=0.))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=10, top_p=0.))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=50, top_p=0.))
print('-'*32)
print(generate_sent('사랑해', gpt_model, top_k=100, top_p=0.))
print('-'*32)

사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해
--------------------------------
사랑해 내새끼들아 사랑해
--------------------------------


In [ ]:
#결과 확인
print(generate_sent('쓸쓸한 밤', gpt_model, greedy=True))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=0, top_p=0.95))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=0, top_p=0.90))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=0, top_p=0.7))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=0, top_p=0.5))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=5, top_p=0.))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=10, top_p=0.))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=50, top_p=0.))
print('-'*32)
print(generate_sent('쓸쓸한 밤', gpt_model, top_k=100, top_p=0.))
print('-'*32)

쓸쓸한 밤, 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
쓸쓸한 밤
--------------------------------
쓸쓸한 밤, 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
쓸쓸한 밤...너는 어디가고 난 어쩌고/날마다 피곤한데/여보, 아직 집으로 오지 마/울다 보면 언젠가 다시 만날 그때 그 날도 오늘처럼 말해요 햇살이 따사로운 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
쓸쓸한 밤, 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
쓸쓸한 밤...이삿짐 트럭에
--------------------------------
쓸쓸한 밤, 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------
쓸쓸한 밤
--------------------------------
쓸쓸한 밤.
--------------------------------
쓸쓸한 밤길을 이리저리 거닐다 보면 어느새 초로의 정취로 충만해 지는 어느 날 그대와 같이 걷는 어느 날 차라리 그대 돌아오세
--------------------------------


# 2. 셀프 피드백
- 이전 3_easy_model_lstm.ipynb의 결과보다는 훨씬 만족스러운 답을 얻었다. 시간 부족, 능력 부족, 데이터 부족 등의 문제로 인해 더 나은 결과를 산출하지 못한 점은 아쉽다.